In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import nnls

In [2]:
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/proportion_pxt_neh/"
time=31
pw=0.046

In [3]:
data=pd.read_csv(path+"__CPSM_MOUSE.csv")
data=data[data.exp_Time==time]
data=data[data.exp_Time==time].reset_index(drop="True")
data.head()

,Peptied,charge,exp_Time,NEH,e_NEH,pxt,e_pxt,mo,m1,m2,m3,m4,m5,io,i1,i2,i3,i4,i5
0,SMEYcIIEVNAR,2,31,25,22,0.046,0.050,0.40995,0.31550,0.17244,0.07014,0.02338,0.00661,0.131710,0.261299,0.278459,0.183605,0.105282,0.039646
1,IEFEGQSVDFVDPNK,2,31,28,27,0.046,0.050,0.36570,0.34991,0.18540,0.07061,0.02144,0.00548,0.099898,0.215021,0.271999,0.214834,0.134417,0.063831
2,TAVDSGIALLTNFQVTK,2,31,25,26,0.046,0.045,0.35488,0.35097,0.19026,0.07374,0.02269,0.00586,0.112706,0.241696,0.267777,0.206311,0.116566,0.054945
3,EPLFGISTGNIITGLAAGAK,3,31,36,32,0.046,0.048,0.32091,0.34919,0.20558,0.08588,0.02834,0.00782,0.075000,0.185255,0.252018,0.237232,0.162041,0.088454
4,PSYVLSGSAMNVVFSEDEMK,2,31,36,32,0.046,0.048,0.26161,0.31262,0.22573,0.12075,0.05225,0.01916,0.055720,0.167710,0.233795,0.236165,0.186275,0.120335


In [4]:
res=[]
for pep_index in range(data.shape[0]):
    exp_0=list(data.loc[pep_index,["mo","m1","m2","m3","m4","m5"]])
    b=list(data.loc[pep_index,["io","i1","i2","i3","i4","i5"]])#exp_t
    
    A=[]
    for i in range(6):
        A.append(([0]*i)+exp_0[:6-i])
    A=np.array(A).transpose()
#     x=np.matmul( np.linalg.inv ( np.matmul(A.transpose(),A)) , np.matmul(A.transpose(),b)  )
    x=list(nnls(A,b)[0])
    
    neh= (x[2]/x[1])*2* ((1-pw)/pw) +1
    xm=sum([(index+1)*val / sum(x) for index,val in enumerate(x)])
    res.append([data.loc[pep_index,"Peptied"],data.loc[pep_index,"charge"],data.loc[pep_index,"NEH"],neh,xm, 
               data.loc[pep_index,"e_NEH"]*data.loc[pep_index,"e_pxt"]])
#     print(data.loc[pep_index,"Peptied"],x,end="\n\n")

In [5]:
res=pd.DataFrame(res)
res.columns=["Peptide","Charge","True NEH","e NEH","sum_xi","pxt_neh"]
res["dif"]=abs(res["True NEH"]- res["e NEH"])
res["test"]=100*abs(res["sum_xi"]-res["pxt_neh"])/res["pxt_neh"]
res=res.sort_values(by="Peptide")
res

,Peptide,Charge,True NEH,e NEH,sum_xi,pxt_neh,dif,test
41,AADTIGYPVMIR,2,22,19.794626,1.909365,0.920000,2.205374,107.539740
6,AFAISGPFNVQFLVK,2,25,21.254328,2.073809,1.049999,3.745672,97.505711
46,AFAmTNQILVER,2,25,22.753380,2.141740,1.056000,2.246620,102.816299
45,AMLSTGFK,2,11,5.371362,1.400172,0.423000,5.628638,231.010075
15,AVNTLNEALEFANSVGYPcLLR,3,42,39.261974,2.810590,1.749999,2.738026,60.605228
36,DGSIDLVINLPNNNTK,3,23,16.561304,1.947530,0.930999,6.438696,109.187041
32,DGSIDLVINLPNNNTK,2,23,24.085994,2.131078,1.022000,1.085994,108.520311
47,EIEYEVVR,2,17,19.038029,1.861650,0.856000,2.038029,117.482480
11,EPLFGISTGNIITGL,2,22,20.270943,2.063372,0.949000,1.729057,117.425858
3,EPLFGISTGNIITGLAAGAK,3,36,37.493907,2.533843,1.535999,1.493907,64.963806


In [6]:
res.to_csv("testx2x1.csv",index=False)

In [7]:
temp=pd.read_csv("C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/pw_known_search_neh/__CPSM_MOUSE.csv")
temp["dif"]=temp.NEH-temp.e_NEH
temp.sort_values(by="Peptied") 

,Peptied,charge,exp_Time,NEH,e_NEH,pxt,e_pxt,mo,m1,m2,m3,m4,m5,io,i1,i2,i3,i4,i5,dif
41,AADTIGYPVMIR,2,31,22,20,0.046,0.046,0.44601,0.32743,0.15340,0.05365,0.01510,0.00357,0.174130,0.299579,0.263794,0.162795,0.073289,0.026414,2
6,AFAISGPFNVQFLVK,2,31,25,23,0.046,0.046,0.36014,0.35594,0.18777,0.06969,0.02031,0.00492,0.122818,0.263328,0.273634,0.193002,0.102132,0.045086,2
46,AFAmTNQILVER,2,31,25,23,0.046,0.046,0.42678,0.33135,0.16118,0.05842,0.01704,0.00418,0.151725,0.273339,0.259636,0.177228,0.090131,0.047942,2
45,AMLSTGFK,2,31,11,9,0.046,0.046,0.58079,0.27924,0.10414,0.02836,0.00617,0.00112,0.376504,0.368157,0.180956,0.055197,0.014067,0.005119,2
15,AVNTLNEALEFANSVGYPcLLR,3,31,42,38,0.046,0.046,0.23747,0.31708,0.23647,0.12717,0.05445,0.01956,0.043666,0.135908,0.218687,0.244951,0.211285,0.145503,4
36,DGSIDLVINLPNNNTK,3,31,23,20,0.046,0.046,0.37764,0.34911,0.18004,0.06699,0.01991,0.00499,0.138443,0.288834,0.275047,0.177259,0.085084,0.035333,3
32,DGSIDLVINLPNNNTK,2,31,23,22,0.046,0.046,0.37764,0.34911,0.18004,0.06699,0.01991,0.00499,0.141459,0.265212,0.264404,0.189530,0.088244,0.051150,1
47,EIEYEVVR,2,31,17,18,0.046,0.046,0.54748,0.31389,0.10593,0.02634,0.00530,0.00091,0.235652,0.331836,0.242566,0.137067,0.038301,0.014579,-1
11,EPLFGISTGNIITGL,2,31,22,20,0.046,0.046,0.40302,0.35031,0.16821,0.05800,0.01592,0.00368,0.156163,0.286505,0.266272,0.166481,0.080019,0.044561,2
3,EPLFGISTGNIITGLAAGAK,3,31,36,33,0.046,0.046,0.32091,0.34919,0.20558,0.08588,0.02834,0.00782,0.075000,0.185255,0.252018,0.237232,0.162041,0.088454,3
